In [3]:
# Standard library imports
import os
import json
import random
from collections import defaultdict
from pathlib import Path
from typing import List, Dict, Set, Tuple

# Third-party imports
import cv2
import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from datasets import Dataset, DatasetDict, load_dataset

import matplotlib.pyplot as plt

In [4]:
#! pip install opencv-python

In [5]:
def analyze_categories(dataset):
    """
    Analyze clothing categories in a fashion dataset.
    
    Args:
        dataset
        
    Returns:
        pd.DataFrame: Statistics for each category
    """
    
    # Get category names from features
    category_names = dataset['train'].features['objects'].feature['category'].names
    
    # Initialize statistics
    stats = defaultdict(lambda: {
        'count': 0,
        'total_area': 0,
        'areas': []
    })
    
    # Process training set
    print("Analyzing object statistics...")
    for item in tqdm(dataset['train']):
        objects = item['objects']
        for cat_id, area in zip(objects['category'], objects['area']):
            cat_name = category_names[cat_id]
            stats[cat_name]['count'] += 1
            stats[cat_name]['total_area'] += area
            stats[cat_name]['areas'].append(area)
    
    # Calculate statistics and create DataFrame
    data = []
    for cat, metrics in stats.items():
        if metrics['count'] > 0:  # Avoid division by zero
            data.append({
                'category': cat,
                'count': metrics['count'],
                'avg_area': metrics['total_area'] / metrics['count'],
                'area_std': np.std(metrics['areas']),
                'is_part': cat.lower() in {'collar', 'pocket', 'sleeve', 
                                         'zipper', 'button', 'buckle'}
            })
    
    df = pd.DataFrame(data)
    
    # Calculate combined score and sort
    df['score'] = df['count'] * df['avg_area']
    df = df.sort_values('score', ascending=False)
    
    # Filter and display main garments
    main_garments = df[~df['is_part']].head(10)
    print("\nTop 10 Main Garments by frequency and size:")
    print(main_garments[['category', 'count', 'avg_area']].round(2).to_string())
    
    # Visualize
    plt.figure(figsize=(15, 6))
    plt.bar(main_garments['category'], main_garments['avg_area'])
    plt.xticks(rotation=45, ha='right')
    plt.title('Average Area by Category')
    plt.ylabel('Average Area (pixels²)')
    plt.tight_layout()
    plt.show()
    
    return df


In [ ]:
dataset_name="detection-datasets/fashionpedia"
dataset = load_dataset(dataset_name)
print("Dataset loaded!!")

In [ ]:
analyze_categories(dataset)

In [12]:
def select_category_images(
    dataset_dict,
    categories: List[str],
    n_train: int = 100,
    n_val: int = 20,
    seed: int = 42
) -> Tuple[Dataset, Dataset]:
    """
    Select images containing ALL specified categories for training and validation.
    
    Args:
        dataset_dict: HuggingFace DatasetDict
        categories: List of category names to include
        n_train: Number of training images to select
        n_val: Number of validation images to select
        seed: Random seed for reproducibility
        
    Returns:
        Tuple of (training_dataset, validation_dataset)
    """
    random.seed(seed)
    np.random.seed(seed)
    
    # Access the training split
    dataset = dataset_dict['train']
    
    # Get category IDs from names
    category_ids = {
        name: idx for idx, name in enumerate(
            dataset.features['objects'].feature['category'].names
        )
    }
    
    # Convert input categories to IDs
    target_cat_ids = {category_ids[cat] for cat in categories if cat in category_ids}
    
    # Find images containing ALL target categories
    valid_indices = []
    for idx, item in enumerate(dataset):
        cats_in_image = set(item['objects']['category'])
        # Check if ALL target categories are in this image
        if target_cat_ids.issubset(cats_in_image):
            valid_indices.append(idx)
    
    # Ensure we have enough images
    total_needed = n_train + n_val
    if len(valid_indices) < total_needed:
        raise ValueError(
            f"Not enough images with ALL specified categories. "
            f"Found {len(valid_indices)}, need {total_needed}. "
            f"Try reducing n_train and n_val or selecting fewer categories."
        )
    
    # Shuffle indices
    random.shuffle(valid_indices)
    
    # Split into train and val
    train_indices = valid_indices[:n_train]
    val_indices = valid_indices[n_train:n_train + n_val]
    
    # Create new datasets
    train_dataset = dataset.select(train_indices)
    val_dataset = dataset.select(val_indices)
    
    # Print statistics
    print("\nDataset split statistics:")
    print(f"Total images with all categories: {len(valid_indices)}")
    print(f"Training images: {len(train_dataset)}")
    print(f"Validation images: {len(val_dataset)}")
    
    # Print category distribution
    print("\nCategory distribution:")
    for split_name, split_data in [("Train", train_dataset), ("Val", val_dataset)]:
        print(f"\n{split_name} split:")
        cat_counts = defaultdict(int)
        for item in split_data:
            for cat_id in item['objects']['category']:
                if cat_id in target_cat_ids:
                    cat_name = dataset.features['objects'].feature['category'].names[cat_id]
                    cat_counts[cat_name] += 1
        
        for cat in categories:
            print(f"{cat}: {cat_counts[cat]} instances")
    
    return train_dataset, val_dataset


In [13]:
# Trying with fewer categories to find images with all categories
categories_of_interest = [
    "pants",
     "bag, wallet", 
    "shirt, blouse"  # Starting with just two categories as example
]

try:
    train_ds, val_ds = select_category_images(
        dataset,
        categories=categories_of_interest,
        n_train=200,  # Reduced numbers since we need images with ALL categories
        n_val=50
    )
except ValueError as e:
    print(f"Error: {e}")
    print("Try reducing the number of categories or the required images.")


Dataset split statistics:
Total images with all categories: 368
Training images: 200
Validation images: 50

Category distribution:

Train split:
pants: 201 instances
bag, wallet: 215 instances
shirt, blouse: 206 instances

Val split:
pants: 50 instances
bag, wallet: 53 instances
shirt, blouse: 50 instances


In [16]:
def visualize_and_save_dataset(
    dataset,
    categories: List[str],
    save_dir: str,
    num_images: int = 5,
    thickness: int = 2,
    font_scale: float = 0.8
) -> None:
    """
    Visualize images with bounding boxes and labels using OpenCV.
    
    Args:
        dataset: HuggingFace dataset split
        categories: List of category names to highlight
        save_dir: Directory to save visualizations
        num_images: Number of images to visualize
        thickness: Line thickness for bounding boxes
        font_scale: Font scale for labels
    """
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Get category IDs
    category_ids = {
        name: idx for idx, name in enumerate(
            dataset.features['objects'].feature['category'].names
        )
    }
    target_cat_ids = {category_ids[cat] for cat in categories}
    
    # Create color map for categories (BGR format for OpenCV)
    np.random.seed(42)  # for reproducible colors
    colors = np.random.randint(0, 255, size=(len(categories), 3)).tolist()
    color_map = dict(zip(categories, colors))
    
    # Font for OpenCV
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Process specified number of images
    for i in range(min(num_images, len(dataset))):
        try:
            example = dataset[i]
            
            # Convert PIL image to numpy array if necessary
            image = np.array(example['image'])
            
            # Convert to BGR format for OpenCV
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            objects = example['objects']
            
            # Get image dimensions
            img_height, img_width = image.shape[:2]
            print(f"Processing image {i+1}, shape: {image.shape}")
            
            # Create a copy of the image to draw on
            img_draw = image.copy()
            
            # Draw bounding boxes and labels
            for bbox, cat_id in zip(objects['bbox'], objects['category']):
                if cat_id in target_cat_ids:
                    cat_name = dataset.features['objects'].feature['category'].names[cat_id]
                    
                    # Get bbox coordinates
                    x1, y1, x2, y2 = map(int, bbox)  # Convert to integers for OpenCV
                    
                    # Verify if bbox is within reasonable bounds
                    if (x1 < 0 or y1 < 0 or 
                        x2 > img_width or 
                        y2 > img_height or 
                        x2-x1 <= 0 or y2-y1 <= 0):
                        print(f"Warning: Invalid bbox for {cat_name} in image {i+1}")
                        print(f'Box is {xy},{y1}, {x2}, {y2} for image w {img_width}, {img_height}')
                        continue
                    
                    # Get color for this category
                    color = color_map[cat_name]
                    
                    # Draw rectangle
                    cv2.rectangle(img_draw, (x1, y1), (x2, y2), color, thickness)
                    
                    # Add label with background
                    label = cat_name
                    (label_w, label_h), _ = cv2.getTextSize(label, font, font_scale, thickness)
                    
                    # Draw label background
                    label_y = max(y1 - label_h - 10, 0)  # Keep label above box but within image
                    cv2.rectangle(
                        img_draw,
                        (x1, label_y),
                        (x1 + label_w, label_y + label_h + 10),
                        color,
                        -1  # Filled rectangle
                    )
                    
                    # Draw label text (white)
                    cv2.putText(
                        img_draw,
                        label,
                        (x1, label_y + label_h + 5),
                        font,
                        font_scale,
                        (255, 255, 255),  # White text
                        thickness
                    )
            
            # Add title (black text with white background)
            title = f'Image {i+1}'
            (title_w, title_h), _ = cv2.getTextSize(title, font, 1, thickness)
            
            # Draw title background
            cv2.rectangle(
                img_draw,
                (10, 10),
                (10 + title_w, 10 + title_h + 10),
                (255, 255, 255),
                -1
            )
            
            # Draw title text
            cv2.putText(
                img_draw,
                title,
                (10, 10 + title_h + 5),
                font,
                1,
                (0, 0, 0),  # Black text
                thickness
            )
            
            # Save image
            save_path = os.path.join(save_dir, f'image_{i+1:04d}.jpg')
            cv2.imwrite(save_path, img_draw)
            
            if (i + 1) % 10 == 0:
                print(f"Processed {i+1} images")
                
        except Exception as e:
            print(f"Error processing image {i+1}: {str(e)}")
            import traceback
            print(traceback.format_exc())
            continue
    
    print(f"Completed! Saved {min(num_images, len(dataset))} visualizations to {save_dir}")

In [19]:
# Visualize and save validation images
print("\nSaving validation set visualizations...")
visualize_and_save_dataset(
    train_ds,
    categories_of_interest,
    num_images=250,
    save_dir='train_visualizations'
)


Saving validation set visualizations...
Processing image 1, shape: (731, 1024, 3)
Processing image 2, shape: (1024, 682, 3)
Processing image 3, shape: (1024, 768, 3)
Processing image 4, shape: (1024, 682, 3)
Processing image 5, shape: (1024, 682, 3)
Processing image 6, shape: (1024, 743, 3)
Processing image 7, shape: (1024, 682, 3)
Processing image 8, shape: (1024, 559, 3)
Processing image 9, shape: (1024, 1024, 3)
Processing image 10, shape: (1024, 502, 3)
Processed 10 images
Processing image 11, shape: (1024, 806, 3)
Processing image 12, shape: (683, 1024, 3)
Processing image 13, shape: (1024, 1024, 3)
Processing image 14, shape: (1024, 682, 3)
Processing image 15, shape: (1024, 682, 3)
Processing image 16, shape: (1024, 681, 3)
Processing image 17, shape: (1024, 695, 3)
Processing image 18, shape: (1024, 681, 3)
Processing image 19, shape: (1024, 682, 3)
Processing image 20, shape: (683, 1024, 3)
Processed 20 images
Processing image 21, shape: (1024, 1024, 3)
Processing image 22, s

In [1]:
def simplify_category_name(cat_name: str) -> str:
    """
    Simplify category names by removing commas and taking first term.
    Example: 'bag, wallet' -> 'bag'
            'shirt, blouse' -> 'shirt'
    """
    return cat_name.split(',')[0].strip()

def convert_and_save_dataset(
    dataset,
    categories: List[str],
    save_dir: str,
    split_name: str
) -> None:
    """
    Convert dataset to specified format with simplified category names.
    """
    # Create save directories
    save_dir = Path(save_dir)
    images_dir = save_dir / 'images' / split_name
    images_dir.mkdir(parents=True, exist_ok=True)
    
    # Get category IDs with simplified names
    category_ids = {
        name: idx for idx, name in enumerate(
            dataset.features['objects'].feature['category'].names
        )
    }
    # Create mapping between original and simplified names
    target_cat_ids = {
        category_ids[cat]: simplify_category_name(cat) 
        for cat in categories if cat in category_ids
    }
    
    # Prepare annotation file
    annotations_file = save_dir / f'{split_name}_annotations.txt'
    
    # Process dataset
    print(f"Processing {split_name} split...")
    with open(annotations_file, 'w') as f:
        for idx, example in enumerate(tqdm(dataset)):
            image = np.array(example['image'])
            img_height, img_width = image.shape[:2]
            
            # Generate image filename
            image_filename = f'{split_name}_{idx:06d}.jpg'
            
            # Save image
            image_path = images_dir / image_filename
            image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(image_path), image_bgr)
            
            # Process annotations
            for cat_id, bbox in zip(example['objects']['category'], 
                                  example['objects']['bbox']):
                if cat_id in target_cat_ids:
                    # Use simplified category name
                    simple_cat_name = target_cat_ids[cat_id]
                    x, y, w, h = map(int, bbox)
                    
                    # Write annotation line with simplified category name
                    annotation_line = (f'{simple_cat_name} {x} {y} {w} {h} '
                                     f'{image_filename} {img_width} {img_height}\n')
                    f.write(annotation_line)

    print(f"Completed processing {split_name} split!")
    print(f"Images saved to: {images_dir}")
    print(f"Annotations saved to: {annotations_file}")


NameError: name 'List' is not defined

In [ ]:
try:
    # Convert and save both splits
    save_dir = Path('fashion_dataset')

    # Process training split
    convert_and_save_dataset(
        train_ds,
        categories=categories_of_interest,
        save_dir=save_dir,
        split_name='train'
    )
except:
    